it may be worth checking these https://huggingface.co/MBZUAI/LaMini-Flan-T5-248M , https://huggingface.co/Qwen/Qwen2.5-0.5B-Instruct

In [1]:
!pip install python-dotenv bitsandbytes "numpy<2" "transformers>=4.45.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import sys
import warnings
warnings.filterwarnings("ignore")
from pprint import pprint as pp
from dotenv import load_dotenv
import os
from huggingface_hub import whoami, HfFolder


In [2]:
def log_hf():

    load_dotenv("env_vars.env")
    hf_token = os.environ.get("HF_ACCESS_TOKEN")
    HfFolder.save_token(hf_token)
    return print(whoami()["name"])


In [3]:
log_hf()
load_dotenv("env_vars.env")


alberto-lorente


True

In [3]:
df = pd.read_csv("df_from_exp_to_imp.csv")

In [4]:
df.head()

,clean_post,post,class,implicit_class,extra_implicit_class,target,implied_statement,split,time
0,university of washington ( seattle wa ) #proj...,university of washington ( seattle wa ) #proj...,not_hate,NaN,NaN,NaN,NaN,validation,2
1,no excuse not to - neoliberal media / establis...,: no excuse not to - neoliberal media / establ...,not_hate,NaN,NaN,NaN,NaN,train,2
2,human being is a white termanimals ... are wha...,human being is a white termanimals ... are wha...,explicit_hate,NaN,NaN,NaN,NaN,validation,1
3,youre aware that australians are white right ...,youre aware that australians are white right ...,not_hate,NaN,NaN,NaN,NaN,train,2
4,and those who are too stupid to realize or car...,and those who are too stupid to realize or car...,not_hate,NaN,NaN,NaN,NaN,train,1


In [5]:
df["class"].unique()

array(['not_hate', 'explicit_hate', 'implicit_hate'], dtype=object)

### Attaching the prompt to the clean post

In [6]:
base_prompt = """You are a social media content moderator.
INSTRUCTION: The following is a social media message that needs to be classified with the label HATEFUL or NOT HATEFUL.
MESSAGE: {}
OUTPUT AND FORMAT: your output should be just the label."""

In [7]:
def format_prompt(text, base_prompt=base_prompt):

    formatted_prompt = base_prompt.format(text)

    return formatted_prompt

In [8]:
df["formatted_prompt"] = df["clean_post"].apply(format_prompt)

In [9]:
df["formatted_prompt"][0:3]

0    You are a social media content moderator.\nINS...
1    You are a social media content moderator.\nINS...
2    You are a social media content moderator.\nINS...
Name: formatted_prompt, dtype: object

In [10]:
def translate_class_to_label(class_):

    translation_dict = {"not_hate": "NOT HATEFUL",
                        "explicit_hate": "HATEFUL",
                        "implicit_hate": "HATEFUL"}

    translated_label = translation_dict[class_]

    return translated_label


In [11]:
df["label"] = df["class"].apply(translate_class_to_label)

In [12]:
df["label"][0:3]

0    NOT HATEFUL
1    NOT HATEFUL
2        HATEFUL
Name: label, dtype: object

In [13]:
def format_message(formatted_prompt, label=True):
    if label:
        messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": formatted_prompt},
            {"role": "assistant", "content": label}
        ]
    else:
        messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": formatted_prompt}
        ]
    return messages

### Turning the Df into a DatasetDict

In [14]:
from datasets import Dataset, DatasetDict

t_1 = []
t_2 = []

for split in df["split"].unique():

    split_df_1 = df[(df["split"] == split) & (df["time"] == 1)]
    split_df_2 = df[(df["split"] == split) & (df["time"] == 2)]

    hf_split_1 = Dataset.from_pandas(split_df_1)
    hf_split_2 = Dataset.from_pandas(split_df_2)

    t_1.append(hf_split_1)
    t_2.append(hf_split_2)

hf_time_1 = DatasetDict({t_1[0]["split"][0]: t_1[0],
                        t_1[1]["split"][0]: t_1[1],
                        t_1[2]["split"][0]: t_1[2]})

hf_time_2 = DatasetDict({t_2[0]["split"][0]: t_2[0],
                        t_2[1]["split"][0]: t_2[1],
                        t_2[2]["split"][0]: t_2[2]})

## Tokenizer Work

In [15]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    set_seed,
    Seq2SeqTrainer,
    BitsAndBytesConfig,
    LlamaTokenizer
)

In [16]:
set_seed(42)

In [17]:
model_id = "Qwen/Qwen2.5-0.5B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [18]:
tokenizer.pad_token = tokenizer.eos_token

In [20]:
        # padding (`bool`, defaults to `False`):
        #     Whether to pad sequences to the maximum length. Has no effect if tokenize is `False`.
        # truncation (`bool`, defaults to `False`):
        #     Whether to truncate sequences at the maximum length. Has no effect if tokenize is `False`.
        # max_length (`int`, *optional*):
        #     Maximum length (in tokens) to use for padding or truncation. Has no effect if tokenize is `False`. If
        #     not specified, the tokenizer's `max_length` attribute will be used as a default.


In [21]:
def preprocess_and_tokenize(formatted_prompt, label=True, add_generation_prompt=False, context_length=512, output_messages_list=False):

    messages = format_message(formatted_prompt, label)
    tokenized = tokenizer.apply_chat_template(messages,
                                                tokenize=True,
                                                add_generation_prompt=add_generation_prompt,
                                                padding="max_length",
                                                truncation=True,
                                                max_length=context_length,
                                                return_dict=True,
                                                return_tensors="pt")
    if output_messages_list:
        return tokenized, messages

    return tokenized

In [22]:
example = preprocess_and_tokenize(hf_time_1["train"]["formatted_prompt"][0], hf_time_1["train"]["label"][0])
# example

In [23]:
hf_time_1 = hf_time_1.map(preprocess_and_tokenize, input_columns=["formatted_prompt", "label"], batched=False)
hf_time_2 = hf_time_2.map(preprocess_and_tokenize, input_columns=["formatted_prompt", "label"], batched=False)

Map:   0%|          | 0/718 [00:00<?, ? examples/s]

Map:   0%|          | 0/5752 [00:00<?, ? examples/s]

Map:   0%|          | 0/720 [00:00<?, ? examples/s]

Map:   0%|          | 0/1019 [00:00<?, ? examples/s]

Map:   0%|          | 0/8155 [00:00<?, ? examples/s]

Map:   0%|          | 0/1020 [00:00<?, ? examples/s]

In [24]:
hf_time_1.set_format("torch")
hf_time_2.set_format("torch")

In [25]:
hf_time_1["train"].column_names

['clean_post',
 'post',
 'class',
 'implicit_class',
 'extra_implicit_class',
 'target',
 'implied_statement',
 'split',
 'time',
 'formatted_prompt',
 'label',
 '__index_level_0__',
 'input_ids',
 'attention_mask']

In [26]:
cols_to_remove = ["clean_post", "post", "class", "implicit_class", "extra_implicit_class", "target", "implied_statement", "split", "time", "formatted_prompt", "label", "__index_level_0__"]

In [27]:
for split in hf_time_1:
    if split != "test":
        hf_time_1[split] = hf_time_1[split].remove_columns(cols_to_remove)
        hf_time_2[split] = hf_time_2[split].remove_columns(cols_to_remove)

In [28]:
hf_time_1["train"].column_names

['input_ids', 'attention_mask']

In [29]:
from transformers import DataCollatorForLanguageModeling
from torch.utils.data.dataloader import DataLoader

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [30]:
batch_size = 4

In [31]:
hf_time_1_train_loader = DataLoader(hf_time_1["train"], collate_fn=data_collator, batch_size=batch_size)
hf_time_1_validation_loader = DataLoader(hf_time_1["validation"], collate_fn=data_collator, batch_size=batch_size)
hf_time_1_test_loader = DataLoader(hf_time_1["test"], collate_fn=data_collator, batch_size=batch_size)

In [32]:
hf_time_2_train_loader = DataLoader(hf_time_2["train"], collate_fn=data_collator, batch_size=batch_size)
hf_time_2_validation_loader = DataLoader(hf_time_2["validation"], collate_fn=data_collator, batch_size=batch_size)
hf_time_2_test_loader = DataLoader(hf_time_2["test"], collate_fn=data_collator, batch_size=batch_size)

### So far, created the prompt, did the messages with the prompt and answer in place. Applied to chat template and tokenized

## Model Stuff

In [19]:
from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
                                load_in_4bit= True,
                                bnb_4bit_quant_type= "nf4",
                                bnb_4bit_compute_dtype= torch.bfloat16,
                                bnb_4bit_use_double_quant= True,
                                # available_devices = "cuda",
                                # bnb_supported_devices = "cuda"
                            )


In [20]:
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            torch_dtype=torch.bfloat16,
                                            device_map="auto",
                                            quantization_config=bnb_config
                                            )


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


FIX THIS LOSS

In [21]:
from torch.nn import CrossEntropyLoss
import torch

# to deal with the fact that we dont make the first token prediction

def keytoken_weighted_loss(inputs, logits):
    # Shift so that tokens < n predict n
    shift_labels = inputs[..., 1:].contiguous()
    shift_logits = logits[..., :-1, :].contiguous()
    # Calculate per-token loss
    loss_fct = CrossEntropyLoss() # removed the reduction=None
    loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
    # Resize and average loss per sample
    loss_per_sample = loss.view(shift_logits.size(0), shift_logits.size(1)).mean(axis=1)

    return loss_per_sample


In [36]:
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear4bit(in_features=896, out_features=896, bias=True)
          (k_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (v_proj): Linear4bit(in_features=896, out_features=128, bias=True)
          (o_proj): Linear4bit(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear4bit(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear4bit(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (

In [37]:
for param in model.base_model.parameters():
    param.requires_grad = False

In [38]:
model_size = sum(t.numel() for t in model.parameters() if t.requires_grad)
model_size

0

In [39]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
    lora_dropout=0.1,
    bias="none",
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 1,081,344 || all params: 495,114,112 || trainable%: 0.2184


In [40]:
model_size = sum(t.numel() for t in model.parameters() if t.requires_grad)
model_size

1081344

In [41]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
model.to(device)

cuda


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 896)
        (layers): ModuleList(
          (0-23): 24 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=896, out_features=896, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=896, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=896, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4

In [42]:
from torch.optim import AdamW

lr = 1e-5
optimizer = AdamW((param for param in model.parameters() if param.requires_grad), lr=lr)


In [47]:
from torch.nn import CrossEntropyLoss

loss_fn = CrossEntropyLoss()

In [43]:
import gc

In [48]:
n_epochs = 2

for epoch in range(n_epochs):
    torch.cuda.empty_cache()
    gc.collect()
    model.train()

    print("Epoch: ", epoch)
    losses = []

    for i, batch in enumerate(hf_time_1_train_loader):
        if i > 0:
            continue

        torch.cuda.empty_cache()
        gc.collect()

        print("\tBatch: ", i)
        # print(batch)
        batch.to(device)
        # print(batch.keys())
        # print(batch["input_ids"].shape)
        # print(batch["attention_mask"].shape)
        # print(batch["labels"].shape)


        batch = {k:torch.squeeze(v) for k,v in batch.items()}

        # print(batch["input_ids"].shape)
        # print(batch["attention_mask"].shape)
        # print(batch["labels"].shape)


        output = model(**batch)
        logits = output.logits
        loss = loss_fn(logits, batch["labels"])

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        losses.append(loss.detach().item())

        print(batch.keys())
        print(loss.detach().item())
        print(output.logits.shape)
        print(output.probas)

    epoch_loss = sum(losses)/len(hf_time_1_train_loader)
    print(f"Epoch {epoch} Loss: {epoch_loss}")

    model.eval()
    with torch.no_grad():

        torch.cuda.empty_cache()
        gc.collect()

        val_losses = []
        for i, batch in enumerate(hf_time_1_validation_loader):
            if i > 0:
                continue
            batch.to(device)
            batch = {k:torch.squeeze(v) for k,v in batch.items()}

            output = model(**batch)
            logits = output.logits
            val_loss = keytoken_weighted_loss(batch["input_ids"], logits)

            val_losses.append(val_loss.detach().item())
        val_loss_epoch = sum(val_losses)/len(hf_time_1_validation_loader)
        print(f"Epoch {epoch} Validation Loss: {val_loss_epoch}")

Epoch:  0
	Batch:  0


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.16 GiB. GPU 0 has a total capacity of 14.74 GiB of which 270.12 MiB is free. Process 14854 has 14.47 GiB memory in use. Of the allocated memory 13.80 GiB is allocated by PyTorch, and 566.66 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
preds = []

model.eval()
with torch.no_grad():
  for i, test_batch in enumerate(hf_time_1["test"]):

      if i > 0:
          break

      text = test_batch["formatted_prompt"]
      tokenized_chat_template, messages_list = preprocess_and_tokenize(text, label=False, add_generation_prompt=True, output_messages_list=True)
      output = model.generate(**tokenized_chat_template.to(device))
      pred = tokenizer.decode(output[0], skip_special_tokens=True)

      print(text)
      print(tokenized_chat_template)
      print(output)
      print(pred)
      preds.append(pred)

In [ ]:
messages_list

In [ ]:
# print(tokenized_chat_template)
# print(output)

print(type(output))
output.shape # batch_size x length

In [24]:
experiment_name = ""
model_name = model_id.split("/")[-1]
model.save_pretrained(f"alberto-lorente/{model_name}_{experiment_name}", from_pt=True)

In [ ]:
print("Successful Run")

In [ ]:
!jupyter nbconvert TinyLlama.ipynb --to python


In [ ]:
Sequence classification should work with peft, just make sure to pass task_type=TaskType.SEQ_CLS to LoraConfig and make sure to put the scores head inside modules_to_save in LoraConfig as well.

